In [16]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [24]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [25]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [26]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [27]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [28]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [29]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [30]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(5):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-c1f1659cc77044c2a2eb47006fe24144","object":"text_completion","created":443993,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.829922 seconds 


Prompt==> Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-eb7a0fad1ec94fd28b635e7f34178e5c","object":"text_completion","created":443993,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/sna

Prompt==> Problem: Write a function in java to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-46d3e2924e254dd69f50b96253bdd9a2","object":"text_completion","created":444004,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\npublic class Main {\n    public static void main(String[] args) {\n        String s = \"Hello World\";\n        System.out.println(reverseString(s));\n    }\n\n    public static String reverseString(String s) {\n        String reversed = \"\";\n        for (int i = s.length() - 1; i >= 0; i--) {\n            reversed += s.charAt(i);\n        }\n        return reversed;\n    }\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":146,"completion_tokens":115}}
Execution time: 5.932359 seconds 


Prompt==> Problem: Write a function in java to calculate the factorial of a given non-negative integ

Prompt==> Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-688553435c2a4cc18b7769b41ef45af4","object":"text_completion","created":444017,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```java\npublic class BinarySearch {\n    public static int binarySearch(int[] arr, int target) {\n        int left = 0;\n        int right = arr.length - 1;\n\n        while (left <= right) {\n            int mid = left + (right - left) / 2;\n\n            if (arr[mid] == target) {\n                return mid;\n            } else if (arr[mid] < target) {\n                left = mid + 1;\n            } else {\n                right = mid - 1;\n            }\n        }\n\n        return -1;\n    }\n\n    public static void main(String[] args) {\n        int[

Prompt==> Problem: Write a function in c++ to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-57f9f9fef1e64532a0175862db5371f4","object":"text_completion","created":444031,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\nHere is a simple C++ function to check if two strings are anagrams:\n\n```cpp\n#include <bits/stdc++.h>\nusing namespace std;\n\nbool areAnagram(string s1, string s2) {\n    int count[256] = {0};\n    int i;\n\n    if (s1.length() != s2.length())\n        return false;\n\n    for (i = 0; s1[i] && s2[i]; i++) {\n        count[s1[i]]++;\n        count[s2[i]]--;\n    }\n\n    for (i = 0; i < 256; i++)\n        if (count","logprobs":null,"finish_r

Prompt==> Problem: Write a function in javascript to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-10e34252a68a4775b7f74d8d6e4aa8b3","object":"text_completion","created":444047,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction sumOfDigits(num) {\n    let sum = 0;\n    while (num > 0) {\n        sum += num % 10;\n        num = Math.floor(num / 10);\n    }\n    return sum;\n}\n\nconsole.log(sumOfDigits(1234)); // Output: 10\nconsole.log(sumOfDigits(5678)); // Output: 26\nconsole.log(sumOfDigits(9012)); // Output: 4\nconsole.log(sumOfDigits(1000)); // Output: 1\nconsole.log(sumOfDigits(1111)); // Output: 4","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":43,"total_tokens":209,"completion_tokens":166}}
Execution time: 8.729243 seconds 


Prompt==> Problem: Write a fun

...

Prompt==> Problem: Implement a function in CSS to validate an email address.Input: String, email address.Output: Boolean, indicating whether the email address is valid. 
 {"id":"cmpl-f81d77f2d66c4df4985f1e8319079a36","object":"text_completion","created":444106,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function to validate an email address:\n\n```css\n@function is-valid-email($email) {\n    @return (string-length($email) > 0) and (string-length($email) <= 254) and (string-length($email) <= 64) and (string-length($email) <= 128) and (string-length($email) <= 256) and (string-length($email) <= 512) and (string-length($email) <= 1024) and (string-length($email) <= 2048) and (string-length($email) <= 4096) and (string-length($email) <= 8192) and (string-length($email) <= 16384) and (string-length($email","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_toke

Prompt==> Problem: Create an image slider in react that displays images in a sequence with next and previous buttons.Input: Array of image URLs.Output: None (update the displayed image). 
 {"id":"cmpl-94b7c281e30a4be99c6408f1e0bb9a45","object":"text_completion","created":444120,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple implementation of an image slider in React using hooks:\n\n```jsx\nimport React, { useState } from 'react';\n\nconst ImageSlider = ({ images }) => {\n  const [currentIndex, setCurrentIndex] = useState(0);\n\n  const goToPrevious = () => {\n    const isFirstSlide = currentIndex === 0;\n    const newIndex = isFirstSlide ? images.length - 1 : currentIndex - 1;\n    setCurrentIndex(newIndex);\n  };\n\n  const goToNext = () => {\n    const isLastSlide = currentIndex === images.length - 1;\n    const newIndex = isLastSlide ? 0 : currentIndex + 1;\n    setCur

Prompt==> Problem: Build a function in angular to toggle between light and dark modes for a website.Input: None.Output: None (update the website theme). 
 {"id":"cmpl-a9c32dca449144d5add36cc5492bbcea","object":"text_completion","created":444133,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\nIn Angular, you can use a service to manage the theme. Here's a simple example:\n\n1. Create a new service named `ThemeService`:\n\n```typescript\nimport { Injectable } from '@angular/core';\n\n@Injectable({\n  providedIn: 'root'\n})\nexport class ThemeService {\n  private activeTheme = 'light';\n\n  constructor() { }\n\n  getActiveTheme() {\n    return this.activeTheme;\n  }\n\n  setActiveTheme(theme: string) {\n    if (['light', 'dark'].includes(theme)) {\n      this.activeTheme = theme;\n    } else {\n      console.error('Invalid theme');\n    }\n  }\n}\n```\n\n2. Inject the `ThemeServi

Prompt==> Problem: Create a function in angular to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element). 
 {"id":"cmpl-5adbaf3f0f7a485c96092e8ef694931b","object":"text_completion","created":444148,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nExample:\n\nHTML:\n\n```html\n<button (click)=\"toggleClass('myElement', 'myClass')\">Toggle Class</button>\n<div id=\"myElement\">Hello World</div>\n```\n\nAngular Component:\n\n```typescript\nimport { Component } from '@angular/core';\n\n@Component({\n  selector: 'app-root',\n  templateUrl: './app.component.html',\n  styleUrls: ['./app.component.css']\n})\nexport class AppComponent {\n  toggleClass(elementId: string, className: string) {\n    const element = document.getElementById(elementId);\n    if (element) {\n      element.classList.toggl

Prompt==> Problem: Create an image slider in jQuery that displays images in a sequence with next and previous buttons.Input: Array of image URLs.Output: None (update the displayed image). 
 {"id":"cmpl-7c2f71a0220b4f71a9d179f5f8b30d56","object":"text_completion","created":444162,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple implementation of an image slider using jQuery:\n\n```html\n<!DOCTYPE html>\n<html>\n<head>\n    <script src=\"https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js\"></script>\n    <style>\n        #slider {\n            width: 500px;\n            height: 300px;\n            overflow: hidden;\n            position: relative;\n        }\n        #slider img {\n            position: absolute;\n            top: 0;\n            left: 0;\n            opacity: 0;\n            transition: opacity 1s;\n        }\n        #slider img.active {\n    

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))